# Preprocessing

We add spectral indices and VV/VH ratio to the dataset for training.

Various scripts for spectral indices can be found here: 
https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel/sentinel-2/

In [ ]:
import rioxarray as rxr
import geopandas as gpd
import matplotlib.pyplot as plt
import earthpy.plot as ep
import earthpy.spatial as es
import xarray as xr
import numpy as np

def norm_diff(b1, b2):
    return (b1 - b2) / (b1 + b2)

In [ ]:
## Define function that prepares all of the bands

year_dict = {
    '2019': 0,
    '2020': 1,
    '2021': 2,
    '2022': 3,
}

def process_year(year):
    '''
    Param year: The year that will be processed.
    
    Returns: Geotiff in the outputs folder - ghana_prepped_{year}.tif
    Contains all sentinel bands 1-8, 8a, 9, 11, 12
    Contains NDVI, MNDWI, BSI
    Contains VV, VH, and 1 - (VV/(VV+VH))
    '''
    
    # Get spectral data
    ds = rxr.open_rasterio(f'raw_data/ghana_{year}_studyarea.tif', masked = True)

    # Get Radar data
    with rxr.open_rasterio('raw_data/ghana_VV_collection.tif', masked = True) as vv:
        vv_ds = vv[year_dict[year]]

    with rxr.open_rasterio('raw_data/ghana_VH_collection.tif', masked = True) as vh:
        vh_ds = vh[year_dict[year]]
        
    # norm_diff NIR/Red
    ndvi = norm_diff(ds[4], ds[3])

    # norm_diff Green/SWIR
    mndwi = norm_diff(ds[2], ds[10])

    # BSI = ((B11 + B04) - (B08 + B02)) / ((B11 + B04) + (B08 + B02))
    # from https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/barren_soil/
    bsi = 2.5 * norm_diff((ds[10] + ds[3]), (ds[6] + ds[1]))
    
    # Create m
    dop = vv_ds / (vv_ds + vh_ds)
    m = 1 - dop
    
    # Clip m to remove extreme outliers - may wish to remove this step in some cases
    modified_m = m.where(m > np.percentile(m, 0.1), np.percentile(m, 0.1))
    modified_m = modified_m.where(m < np.percentile(m, 99.9), np.percentile(m, 99.9))
    
    # Create final_output xarray
    # Note that we add band coords to the spectral indices and remove SCL band from sentinel data
    final_output = xr.concat([ds[:-1] / 10000, ndvi.assign_coords({'band': 1}), 
                              mndwi.assign_coords({'band': 1}), 
                              bsi.assign_coords({'band': 1}), 
                              vv_ds, vh_ds, modified_m], dim = 'band')
    
    final_output.rio.set_nodata(np.nan)
    final_output.rio.to_raster(f'outputs/ghana_prepped_{year}.tif')

In [ ]:
%%time
process_year('2019')

In [ ]:
ds = rxr.open_rasterio('raw_data/ghana_2019_studyarea.tif', masked = True)

# Note that sentinel-1 data is already preprocessed
# Terrain correction using SRTM 30 or ASTER DEM for areas greater than 60 degrees latitude, where SRTM is not available. 
# The final terrain corrected values are converted to decibels via log scaling (10*log10(x).
# In other words, VV and VH are already log scaled
with rxr.open_rasterio('raw_data/ghana_VV_collection.tif', masked = True) as vv:
    vv_ds = vv[0]
    
with rxr.open_rasterio('raw_data/ghana_VH_collection.tif', masked = True) as vh:
    vh_ds = vh[0]

In [ ]:
ds

In [ ]:
ds = ds[:-1]/10000

ds

In [ ]:
# Create spectral indices
# norm_diff NIR/Red
ndvi = norm_diff(ds[4], ds[3])

# norm_diff Green/SWIR
mndwi = norm_diff(ds[2], ds[10])

# BSI = ((B11 + B04) - (B08 + B02)) / ((B11 + B04) + (B08 + B02))
# from https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/barren_soil/
bsi = 2.5 * norm_diff((ds[10] + ds[3]), (ds[6] + ds[1]))

# # Normalized Difference Built-Up Index computed as SWIR(Band11)-NIR(Band8)/ SWIR(Band11)+NIR(Band8)
# Note that NDBI is very similar to BSI - difficult to distinguish between bare soil and urban areas in many settings
# ndbi = norm_diff(ds[10], ds[7])

ep.plot_bands(bsi, cmap = "RdYlGn")

In [ ]:
# TCC
ep.plot_rgb(ds.data, rgb = (3,2,1), stretch = True)

## Radar

Getting weird results that don't appear to map to correct land use classes. Probably just use VH, VS, and the m, which is related to the relative proportion fo VV to total power. Note that m can have very high and very low values - clip those to 0.1 and 99.9 percentile values.

In [ ]:
# Radar has been used to track vegetation
# This is a good example of a basic script using VV and VH
# https://custom-scripts.sentinel-hub.com/sentinel-1/radar_vegetation_index/
dop = vv_ds / (vv_ds + vh_ds)

# m ranges between 0 and 1, where 0 is urban and 1 is canopy
# however, this is only for pure areas - in cases where there is rough ground (e.g., soil after tillage or rough water)
# then this can give fradulent results. Consequently, need to multiply by vegetation depolarization power fraction
m = 1 - dop

rvi = np.sqrt(m) * (4 * vh_ds) / (vv_ds + vh_ds)

ep.plot_bands(m, cmap = "RdYlGn", vmin = 0.5, vmax = 1)

In [ ]:
print(np.percentile(m, 0.1))
print(np.percentile(m, 99.9))

In [ ]:
# Looks good, but very strongly associated with DEM
modified_m = m.where(m > np.percentile(m, 0.1), np.percentile(m, 0.1))
modified_m = modified_m.where(m < np.percentile(m, 99.9), np.percentile(m, 99.9))

ep.plot_bands(modified_m)

In [ ]:
# # Testing
# test = xr.concat([ds, ndvi.assign_coords({'band': 1}), mndwi.assign_coords({'band': 1}), bsi.assign_coords({'band': 1}), vv_ds, vh_ds, modified_m], dim = 'band')
# test.rio.set_nodata(np.nan)
# test.rio.to_raster('outputs/test.tif')

In [ ]:
# # Working
# test_ds = rxr.open_rasterio('outputs/test.tif', masked = True)

# ep.plot_rgb(test_ds.data, rgb = (3,2,1), stretch = True)

In [ ]:
ds